In [10]:

import pandas as pd
import numpy as np
from datetime import datetime

# Load and examine data files to understand structure
trans_file = "/Users/vijayaraghavandevaraj/Downloads/Wood - TransReport-Book.xlsx"
tax_file = "/Users/vijayaraghavandevaraj/Downloads/Wood - Tax Report.xlsx"

print("📁 Loading Excel files...")
WoodTrans = pd.read_excel(trans_file)
WoodTax = pd.read_excel(tax_file)

print(f"✅ Transaction Report loaded: {len(WoodTrans)} records")
print(f"✅ Tax Report loaded: {len(WoodTax)} records")


📁 Loading Excel files...
✅ Transaction Report loaded: 1913 records
✅ Tax Report loaded: 918 records


In [11]:
#Step 1: Filter the 'trans report' for 'pos' source
trans_pos_df = WoodTrans[WoodTrans['order_type'].isin(['booking'])].copy()
trans_pos_df['Transaction Type'] = 'sale'
trans_pos_df['Module'] = np.where(
    trans_pos_df['Order ID'].astype(str).str.startswith('MEM'),'memberships',trans_pos_df['Source']
)
trans_pos_df.head()

trans_pos_module_df = trans_pos_df[trans_pos_df['Module'].isin(['website', 'crm'])].copy()
trans_pos_module_df.head()

,Location,Order ID,Transaction Date,Amount,Source,Payment Type,Payment Gateway,Transaction Type,Card Type,Currency,Payment ID,Type,order_type,eventDate,Module
58,East Nashville,1757696812642,2025-09-12,29.63,website,virtualCard,authorize,sale,Visa,USD,121235543045,credit,booking,NaT,website
170,East Nashville,1754423645239,2025-09-08,100.00,crm,physicalCard,square,sale,NaN,USD,5ifULCBOztKXWPuUSgFJyY0AG2XZY,debit,booking,NaT,crm
171,East Nashville,1754423645239,2025-09-08,100.00,crm,physicalCard,square,sale,NaN,USD,5ifULCBOztKXWPuUSgFJyY0AG2XZY,debit,booking,NaT,crm
307,East Nashville,1756933898459,2025-09-03,29.63,website,virtualCard,authorize,sale,MasterCard,USD,121219486752,credit,booking,2025-09-04,website
317,East Nashville,1756924876900,2025-09-03,51.58,website,virtualCard,authorize,sale,Visa,USD,121219175726,credit,booking,NaT,website


In [12]:
#Step 1: Filter the 'trans report' for 'pos' source
WoodTax.head()
tax_booking_df = WoodTax[WoodTax['Module Name'] == 'booking'].copy()
tax_booking_df = tax_booking_df[
    (WoodTax['Order Status'] == 'booked') & (WoodTax['Payment Status'] == 'completed')
].copy()
tax_booking_df.head()

/var/folders/ds/dk2692515bbgk_t5gxbfq_sh0000gn/T/ipykernel_98836/694661598.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tax_booking_df = tax_booking_df[


,Order ID,Module Name,Location,Month,Date,Year,Order Status,Payment Status,Tip,Tax,Total Sum
9,1756566442381,booking,East Nashville,Aug,2025-08-30 00:00:00,2025,booked,completed,0.0,-5.17,-58.17
10,1754006154910,booking,East Nashville,Aug,2025-08-01 00:00:00,2025,booked,completed,0.0,0.00,0.00
11,1753980996401,booking,East Nashville,Aug,2025-08-02 00:00:00,2025,booked,completed,0.0,0.00,0.00
12,1754404814977,booking,East Nashville,Aug,2025-08-05 00:00:00,2025,booked,completed,0.0,0.00,0.00
13,1754409589812,booking,East Nashville,Aug,2025-08-05 00:00:00,2025,booked,completed,0.0,0.00,0.00


In [ ]:
# Prepare date columns and create Month column
print("🔧 Preparing date columns...")

# Convert date columns to datetime
# Use eventDate for trans_pos_df filtering
trans_pos_df['eventDate'] = pd.to_datetime(trans_pos_df['eventDate'])
trans_pos_df['Month'] = trans_pos_df['eventDate'].dt.to_period('M')

# Keep Transaction Date for other purposes if needed
trans_pos_df['Transaction Date'] = pd.to_datetime(trans_pos_df['Transaction Date'])

tax_booking_df['Date'] = pd.to_datetime(tax_booking_df['Date'])
tax_booking_df['Month'] = tax_booking_df['Date'].dt.to_period('M')

print(f"   ✅ Date columns prepared")

# Define target months - you can modify this list to include any months you want
target_months = ["2025-08", "2025-09"]  # August and September 2025
# You can also use: ["2025-08"] for just August, or ["2025-09"] for just September

# Filter trans_pos_df for target months based on eventDate
trans_filtered = trans_pos_df[trans_pos_df['Month'].astype(str).isin(target_months)]
tax_filtered = tax_booking_df[tax_booking_df['Month'].astype(str).isin(target_months)]

print(f"   • Filtered transactions ({', '.join(target_months)}): {len(trans_filtered):,} records")
print(f"   • Filtered tax records ({', '.join(target_months)}): {len(tax_filtered):,} records")

# Show breakdown by month
print(f"\n📊 Breakdown by month:")
for month in target_months:
    trans_count = len(trans_filtered[trans_filtered['Month'].astype(str) == month])
    tax_count = len(tax_filtered[tax_filtered['Month'].astype(str) == month])
    print(f"   • {month}: {trans_count:,} transactions, {tax_count:,} tax records")


🔧 Preparing date columns...
   ✅ Date columns prepared
   • August transactions: 101 records
   • August tax records: 202 records
